In [1]:
from datasets import load_dataset

/data/pufanyi/anaconda3/anacondabin/envs/live_bench/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_dataset("lmms-lab/LiveBench", "2024-06")

In [3]:
import pandas as pd

df = data["test"].to_pandas()
df = df[df["checker"].notna()]

In [4]:
df.iloc[620]["checker"] is None

False

In [5]:
df["subtask"].value_counts()

subtask
Contextual Analysis     477
Deeper Implications     132
Basic Understanding     118
Further Insights         63
Broader Implications     52
Name: count, dtype: int64

In [6]:
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
top_50_per_subtask = df.groupby("subtask").apply(lambda x: x.nlargest(50, "score"))
top_50_per_subtask.reset_index(drop=True, inplace=True)
len(top_50_per_subtask)

/tmp/ipykernel_303711/5529174.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_50_per_subtask = df.groupby('subtask').apply(lambda x: x.nlargest(50, 'score'))


250

In [8]:
top_50_per_subtask["subtask"].value_counts()

subtask
Basic Understanding     50
Broader Implications    50
Contextual Analysis     50
Deeper Implications     50
Further Insights        50
Name: count, dtype: int64

In [9]:
import numpy as np

np.mean(top_50_per_subtask["score"])

9.276

In [10]:
from datasets import Dataset, Features
import datasets


def gen():
    for d in top_50_per_subtask:
        yield d


data = Dataset.from_pandas(
    top_50_per_subtask,
    features=Features(
        {
            "id": datasets.Value("int32"),
            "images": datasets.Sequence(datasets.Image()),
            "website": datasets.Value("string"),
            "question": datasets.Value("string"),
            "answer": datasets.Value("string"),
            "criteria": datasets.Value("string"),
            "subtask": datasets.Value("string"),
            "data_generator": datasets.Value("string"),
            "checker": datasets.Value("string"),
            "date_time": datasets.Value("string"),
            "screen_shoter": datasets.Value("string"),
            "screen_size": datasets.Value("string"),
            "score": datasets.Value("int32"),
            "reason": datasets.Value("string"),
            "scorer_name": datasets.Value("string"),
        }
    ),
)

In [11]:
data

Dataset({
    features: ['id', 'images', 'website', 'question', 'answer', 'criteria', 'subtask', 'data_generator', 'checker', 'date_time', 'screen_shoter', 'screen_size', 'score', 'reason', 'scorer_name'],
    num_rows: 250
})

In [12]:
data.features

{'id': Value(dtype='int32', id=None),
 'images': Sequence(feature=Image(mode=None, decode=True, id=None), length=-1, id=None),
 'website': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None),
 'criteria': Value(dtype='string', id=None),
 'subtask': Value(dtype='string', id=None),
 'data_generator': Value(dtype='string', id=None),
 'checker': Value(dtype='string', id=None),
 'date_time': Value(dtype='string', id=None),
 'screen_shoter': Value(dtype='string', id=None),
 'screen_size': Value(dtype='string', id=None),
 'score': Value(dtype='int32', id=None),
 'reason': Value(dtype='string', id=None),
 'scorer_name': Value(dtype='string', id=None)}

In [13]:
data.push_to_hub("lmms-lab/LiveBench", "2024-06", split="test")

Uploading the dataset shards: 100%|██████████| 1/1 [00:18<00:00, 18.34s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lmms-lab/LiveBench/commit/89ce4e07cf0988b2305b7227ac2677d481e6112a', commit_message='Upload dataset', commit_description='', oid='89ce4e07cf0988b2305b7227ac2677d481e6112a', pr_url=None, pr_revision=None, pr_num=None)